In [10]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [11]:
#data
df=pd.read_csv('C:\Kate\Python\Authorship Attribution\data\AllTweets.csv')
author1='ScottKelly'
author2='AdamSavage'
df_kk=df.loc[(df['author'] == author1)]
df_hc=df.loc[(df['author'] == author2)]
df=df_kk.append(df_hc,ignore_index=True)
len(df)

6091

In [12]:
import random
#1000 random sample rows for KK
rows = random.sample(df_kk.index, 1000)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(df_hc.index, 1000)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

2000

In [13]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

1964

In [14]:
#http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}
code_pos_map={v: k for k, v in pos_code_map.iteritems()}
#Python 3 inv_map = {v: k for k, v in my_map.items()}

In [15]:
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [16]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [17]:
df['text_pos']=df.apply(lambda x: pos_tags(x['text']), axis=1)

In [18]:
df.ix[:,['author','text','text_pos']].head()

,author,text,text_pos
0,ScottKelly,Day NUM. An interesting band of clouds over So...,NNCGLFMFNN2NFNNLLL
1,ScottKelly,Soyuz final exam tomorrow. NUM hours in the So...,NGLLNMFCNLL2CLL
2,ScottKelly,Music is a huge part of life in space. Here's ...,N6CGLFLFLTGSNLFCNNNLLLN
3,ScottKelly,Post-tropical Cyclone Linda sits off the South...,NGNN6FCNNLNLLN
4,ScottKelly,#EarthArt A swirl of green runs through it. #Y...,1NLFGMFRNLLN


In [19]:
df_features=pd.DataFrame()

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
for a in df.author.unique():
    v = CountVectorizer(analyzer='char',ngram_range=(7, 7))
    ngrams = v.fit_transform(df[df['author'] == a]['text_pos'])
    df_t=pd.DataFrame(
    {'Feature': v.get_feature_names(),
     'Count': list(ngrams.sum(axis=0).flat),
     'Author': a
    })
    #
    df_features=df_features.append(df_t,ignore_index=True)

In [21]:
df_features['Feature_POS']=df_features.apply(lambda x: text_pos_inv_convert(x['Feature']), axis=1)

In [22]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author2].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
13610,AdamSavage,5,fclllll,IN-DT-NN-NN-NN-NN-NN
14710,AdamSavage,5,glfnnnn,JJ-NN-IN-NNP-NNP-NNP-NNP
17278,AdamSavage,4,ncnnnnn,NNP-DT-NNP-NNP-NNP-NNP-NNP
16016,AdamSavage,4,llfnfnn,NN-NN-IN-NNP-IN-NNP-NNP
13512,AdamSavage,4,fcgllnn,IN-DT-JJ-NN-NN-NNP-NNP


In [23]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author1].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
5392,ScottKelly,38,lfnnlln,NN-IN-NNP-NNP-NN-NN-NNP
7231,ScottKelly,32,nfnnlll,NNP-IN-NNP-NNP-NN-NN-NN
7477,ScottKelly,27,nlfnnll,NNP-NN-IN-NNP-NNP-NN-NN
5975,ScottKelly,19,lnlfnnl,NN-NNP-NN-IN-NNP-NNP-NN
5935,ScottKelly,17,lnfnnll,NN-NNP-IN-NNP-NNP-NN-NN


In [24]:
from sklearn.cross_validation import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df.ix[:,['text','text_pos']], df['author'], test_size=0.3, random_state=42)

In [25]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [26]:
ScoreSummaryByModelParams = list()

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [29]:
class TextAndTextCodedExtractor(BaseEstimator, TransformerMixin):
    """Extract the text & text_pos from a tweet in a single pass.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, tweets):
        features=tweets.ix[:,['text_pos','text']].to_records(index=False)

        return features

In [30]:
def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1, cv=10)
    grid_search.fit(twt_train, author_train)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])    
 

In [31]:
f1_union=FeatureUnion(
        transformer_list=[
              # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char')),
            ])),               

        ],
    )

In [32]:
from sklearn.naive_bayes import BernoulliNB
p = {
    'union__char__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__char__tfidf__ngram_range': ((2, 2), (3, 3)), 
    'union__char__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char')

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    7.1s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   25.0s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   55.5s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  2.6min


Best score: 0.846
Best parameters set:
	clf__alpha: 0.0001
	union__char__tfidf__max_df: 0.5
	union__char__tfidf__max_features: None
	union__char__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  2.9min finished


In [33]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word')),
            ])),              

        ],
    )

In [34]:
p = {
    'union__word__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__word__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__word__tfidf__max_features': (None, 5000, 10000, 50000),
    'union__word__tfidf__stop_words': (None, 'english'),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, word')

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    4.1s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   17.4s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   40.4s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  1.8min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  2.7min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  3.6min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  4.6min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed:  5.8min
[Parallel(n_jobs=1)]: Done 4999 tasks       | elapsed:  7.2min
[Parallel(n_jobs=1)]: Done 6049 tasks       | elapsed:  8.7min


Best score: 0.889
Best parameters set:
	clf__alpha: 0.1
	union__word__tfidf__max_df: 0.5
	union__word__tfidf__max_features: None
	union__word__tfidf__ngram_range: (1, 1)
	union__word__tfidf__stop_words: None


[Parallel(n_jobs=1)]: Done 7199 tasks       | elapsed: 10.4min
[Parallel(n_jobs=1)]: Done 7200 out of 7200 | elapsed: 10.4min finished


In [35]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process)),
            ])),              

        ],
    )

In [36]:
p = {
    'union__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__text__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, stemmed words, no stop words')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   13.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   58.7s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  2.1min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  6.1min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  8.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed: 12.2min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed: 15.9min
[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed: 17.8min finished


Best score: 0.885
Best parameters set:
	clf__alpha: 0.1
	union__text__tfidf__max_df: 0.5
	union__text__tfidf__max_features: None
	union__text__tfidf__ngram_range: (1, 1)


In [37]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling pos tag features  from the text_pos
            ('text_pos', Pipeline([
            ('selector', ItemSelector(key='text_pos')),
            ('tfidf',    TfidfVectorizer(analyzer='char')),
            ])),                  

        ],
    )

In [38]:
p = {
    'union__text_pos__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text_pos__tfidf__ngram_range': ((3, 3), (4, 4),(5,5),(6,6),(7,7)), 
    'union__text_pos__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, POS tags')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    2.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   12.2s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   27.7s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:   49.0s
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  1.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  2.5min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  3.3min


Best score: 0.692
Best parameters set:
	clf__alpha: 1
	union__text_pos__tfidf__max_df: 0.5
	union__text_pos__tfidf__max_features: None
	union__text_pos__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed:  3.7min finished


In [39]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
8,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__stop_words: None
4,"Bernoulli Naive Bayes, word",0.888646,\tclf__alpha: 0.1
5,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__max_df: 0.5
6,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__max_features: None
7,"Bernoulli Naive Bayes, word",0.888646,"\tunion__word__tfidf__ngram_range: (1, 1)"
9,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,\tclf__alpha: 0.1
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,"\tunion__text__tfidf__ngram_range: (1, 1)"
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,\tunion__text__tfidf__max_features: None
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,\tunion__text__tfidf__max_df: 0.5
0,"Bernoulli Naive Bayes, char",0.846434,\tclf__alpha: 0.0001


In [40]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [41]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    8.9s


Best score: 0.881
Best parameters set:
	clf__alpha: 0.0001


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   11.0s finished


In [42]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [43]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   17.6s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   21.6s finished


Best score: 0.876
Best parameters set:
	clf__alpha: 0.0001


In [44]:
f3_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),    
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [45]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   19.7s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   24.1s finished


Best score: 0.885
Best parameters set:
	clf__alpha: 0.001


In [46]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [47]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   19.2s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   23.5s finished


Best score: 0.878
Best parameters set:
	clf__alpha: 0.0001


In [48]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [49]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   10.8s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   13.2s finished


Best score: 0.879
Best parameters set:
	clf__alpha: 0.001


In [50]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [51]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f4_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word + POS tag')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   21.2s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   26.0s finished


Best score: 0.886
Best parameters set:
	clf__alpha: 0.001


In [52]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
4,"Bernoulli Naive Bayes, word",0.888646,\tclf__alpha: 0.1
5,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__max_df: 0.5
6,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__max_features: None
7,"Bernoulli Naive Bayes, word",0.888646,"\tunion__word__tfidf__ngram_range: (1, 1)"
8,"Bernoulli Naive Bayes, word",0.888646,\tunion__word__tfidf__stop_words: None
22,"Bernoulli Naive Bayes, char + word + stemmed w...",0.886463,\tclf__alpha: 0.001
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,\tunion__text__tfidf__max_df: 0.5
19,"Bernoulli Naive Bayes, char + word + stemmed word",0.885007,\tclf__alpha: 0.001
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,"\tunion__text__tfidf__ngram_range: (1, 1)"
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.885007,\tunion__text__tfidf__max_features: None


In [72]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp

In [73]:
ScoreSummaryByVector = list()

In [74]:
def PredictionEvaluation(author_test_b,author_predictions_b,comment):
    Precision=precision_score(author_test_b,author_predictions_b)
    print ('Precision: %0.3f' % (Precision))
    Accuracy=accuracy_score(author_test_b,author_predictions_b)
    print ('Accuracy: %0.3f' % (Accuracy))
    Recall=recall_score(author_test_b,author_predictions_b)
    print ('Recall: %0.3f' % (Recall))
    F1=f1_score(author_test_b,author_predictions_b)
    print ('F1: %0.3f' % (F1))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    ROC_AUC=roc_auc_score(author_test_b,author_predictions_b)
    print ('ROC-AUC: %0.3f' % (ROC_AUC))
    ScoreSummaryByVector.append([Precision,Accuracy,Recall,F1,ROC_AUC,comment])

In [75]:
def ModelRun (f_union,model):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    pipeline.fit(twt_train, author_train)
    author_predicted = pipeline.predict(twt_test)
    
    feature_names=list()
    for p in (pipeline.get_params()['union'].transformer_list):
        fn=(p[0],pipeline.get_params()['union'].get_params()[p[0]].get_params()['tfidf'].get_feature_names())
        feature_names.append(fn)
    df_fn=pd.DataFrame()
    for fn in feature_names:
        df_fn= df_fn.append(pd.DataFrame(
        {'FeatureType': fn[0],
         'Feature': fn[1]
        }),
        ignore_index=True)    
    
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test.values)
    author_predicted_b  = lb.fit_transform(author_predicted)
    return (df_fn,pipeline.get_params()['clf'],author_predicted,author_predicted_b, author_test_b)

In [76]:
def most_informative_feature_for_binary_classification(feature_names, classifier):
    class_labels = classifier.classes_

    topnvalues_class0 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values))
    topnvalues_class1 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values), reverse=True)

    topn_df_class0=pd.DataFrame(topnvalues_class0, columns=['Coef','Feature','FeatureType'])
    topn_df_class0['Author']=class_labels[0]
    
    topn_df_class1=pd.DataFrame(topnvalues_class1, columns=['Coef','Feature','FeatureType'])
    topn_df_class1['Author']=class_labels[1]    
    
    topn_df=topn_df_class0.append(topn_df_class1)
    
        
    return topn_df

In [77]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [78]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f2_union,BernoulliNB(alpha=0.0001))

In [79]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word')

Precision: 0.875
Accuracy: 0.890
Recall: 0.908
F1: 0.891
Confussion matrix:
[[259  38]
 [ 27 266]]
ROC-AUC: 0.890


In [80]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [81]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f3_union,BernoulliNB(alpha=0.0001))

In [82]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word+POS tag')

Precision: 0.875
Accuracy: 0.893
Recall: 0.914
F1: 0.894
Confussion matrix:
[[261  38]
 [ 25 266]]
ROC-AUC: 0.893


In [83]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [84]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f4_union,BernoulliNB(alpha=0.0001))

In [85]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+word+stemmed word+POS tag')

Precision: 0.895
Accuracy: 0.907
Recall: 0.922
F1: 0.908
Confussion matrix:
[[263  32]
 [ 23 272]]
ROC-AUC: 0.907


In [86]:
df_ScoreSummaryByVector=DataFrame(ScoreSummaryByVector,columns=['Precision','Accuracy','Recall','F1','ROC-AUC','Vector'])
df_ScoreSummaryByVector.sort_values(['F1'],ascending=False,inplace=True)
df_ScoreSummaryByVector

,Precision,Accuracy,Recall,F1,ROC-AUC,Vector
2,0.894737,0.906780,0.922034,0.908180,0.906780,char+word+stemmed word+POS tag
1,0.875000,0.893220,0.914089,0.894118,0.893500,char+stemmed word+POS tag
0,0.875000,0.889831,0.907850,0.891122,0.889952,char+stemmed word


In [87]:
TopFeatures_df=most_informative_feature_for_binary_classification(feature_names, clf)

In [88]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,AdamSavage,-15.741218,\n(t,-15.741218,______________________________________________...,-15.741218,______________________________________________...,-15.741218,VB-VB-VBG
1,AdamSavage,-15.741218,\n(v,-15.741218,aaron,-15.741218,aaron,-15.741218,VB-VB-WP
2,AdamSavage,-15.741218,\nur,-15.741218,abandoned,-15.741218,abandon,-15.741218,VB-VB-IN
3,AdamSavage,-15.741218,"""c",-15.741218,abbotsford,-15.741218,abbotsford,-15.741218,VB-VB-PDT
4,AdamSavage,-15.741218,"""e",-15.741218,above,-15.741218,abov,-15.741218,VB-VB-PRP
5,AdamSavage,-15.741218,"""f",-15.741218,absolutely,-15.741218,abus,-15.741218,VB-VBG-DT
6,AdamSavage,-15.741218,"""g",-15.741218,abuse,-15.741218,acceler,-15.741218,VB-VBG-IN
7,AdamSavage,-15.741218,"""h",-15.741218,accelerant,-15.741218,action,-15.741218,VB-VBG-NNP
8,AdamSavage,-15.741218,"""l",-15.741218,accelerated,-15.741218,actor,-15.741218,VB-VBG-PRP$
9,AdamSavage,-15.741218,"""m",-15.741218,accomplished,-15.741218,ad,-15.741218,VB-VBN-VBN


In [89]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,ScottKelly,-0.426084,num,-0.534426,twitter,-0.531941,twitter,-0.650345,NNP-NN-NN
1,ScottKelly,-0.476438,wit,-0.544426,com,-0.544426,com,-0.903256,NN-NN-NNP
2,ScottKelly,-0.493007,com,-0.837145,ref,-0.837145,ref,-1.067046,NNP-NNP-NN
3,ScottKelly,-0.500192,tte,-0.962533,the,-0.962533,the,-1.088460,NN-NN-NN
4,ScottKelly,-0.500192,pic,-0.997488,yearinspacepic,-0.997488,yearinspacep,-1.252763,IN-NNP-NNP
5,ScottKelly,-0.512284,itt,-1.283853,to,-1.283853,to,-1.315942,NNP-NNP-NNP
6,ScottKelly,-0.512284,er.,-1.294435,num,-1.294435,num,-1.547271,NN-IN-NNP
7,ScottKelly,-0.526991,twi,-1.349094,from,-1.349094,from,-1.618222,IN-DT-NN
8,ScottKelly,-0.539413,ic.,-1.383383,of,-1.383383,of,-1.830397,NN-NNP-NN
9,ScottKelly,-0.544426,r.c,-1.468282,in,-1.468282,in,-1.886486,JJ-NN-IN


In [90]:
author_predicted=pd.DataFrame(author_predicted,columns=['predicted'])
df_wrong_result = pd.concat([twt_test.reset_index(),author_test.reset_index(),author_predicted], axis=1)
df_wrong_result.drop('index', axis=1, inplace=True)
df_wrong_result.drop('text_pos', axis=1, inplace=True)
df_wrong_result=df_wrong_result[df_wrong_result['author']<>df_wrong_result['predicted']]
df_wrong_result.head(10)

,text,author,predicted
0,We've had to move the REF to avoid debris. One...,ScottKelly,AdamSavage
29,Very cool video on the smart first-down line. ...,AdamSavage,ScottKelly
67,Incredible rescue mission in twin de Havilland...,AdamSavage,ScottKelly
77,REF you are correct!,ScottKelly,AdamSavage
78,I need a haircut. Pretty sure.pic.twitter.com/...,AdamSavage,ScottKelly
84,"Loved this street art in Denver, by David Choe...",AdamSavage,ScottKelly
88,Brian Lam has another beautiful article on und...,AdamSavage,ScottKelly
91,Another pic of the same location. Seriously ca...,AdamSavage,ScottKelly
110,Yes! I have taken pictures of it.URL,ScottKelly,AdamSavage
121,Here are a dozen views of the Sydney Opera Hou...,AdamSavage,ScottKelly
